In [19]:
import cv2
import mediapipe as mp
from matplotlib import pyplot as plt
import numpy as np
from scipy import sparse
import torch
import import_ipynb
import import_ipynb
from localAngle import calc_localAngle
from coordinate import coordinateFeature
from coordinate import normFunction

In [20]:
# Calculating Local Angle For Node Feature
def local_angles(BINARY_ADJACENCY_MATRIX, x_axis, y_axis, z_axis):
    row_indices, col_indices = BINARY_ADJACENCY_MATRIX.nonzero()
    Local_Angle_features=[]
    for landmark in range(468):
        List = []
        for i in range(len(row_indices)):
            if(row_indices[i] == landmark):
                List.append([x_axis[col_indices[i]], y_axis[col_indices[i]], z_axis[col_indices[i]]])
        A = [x_axis[landmark], y_axis[landmark], z_axis[landmark]]
        # print("A=",A, "S=",List)
        Local_Angle_features.append(calc_localAngle(A, np.array(List)))
    # print("Local_Angle_features=",Local_Angle_features)
    return normFunction(Local_Angle_features)

In [21]:
def mediapipe(path): 
    image=cv2.imread(path)
    image=cv2.resize(image,(600,600))
    annoted_image=image.astype(np.uint8)

    mp_face_mesh=mp.solutions.face_mesh
    connection_tesselation=mp_face_mesh.FACEMESH_TESSELATION
    # print("edges_mediapipe=",len(connection_tesselation),len(connection_tesselation)/2)
    # print(connection_tesselation)
    with mp_face_mesh.FaceMesh(static_image_mode=False,max_num_faces=2,refine_landmarks=True,min_detection_confidence=0.5) as face_mesh:
        results=face_mesh.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))

    x_axis=np.empty(len(results.multi_face_landmarks[0].landmark))
    y_axis=np.empty(len(results.multi_face_landmarks[0].landmark))
    z_axis=np.empty(len(results.multi_face_landmarks[0].landmark))
    i=0

    for nodes in results.multi_face_landmarks[0].landmark:
        x_axis[i]=(results.multi_face_landmarks[0].landmark[i].x)
        y_axis[i]=(results.multi_face_landmarks[0].landmark[i].y)
        z_axis[i]=(results.multi_face_landmarks[0].landmark[i].z)
        i=i+1
    # print(len(results.multi_face_landmarks[0].landmark))
    # print("xyz=",x_axis)

    FEATURE_MATRIX=np.zeros((468,468))
    for i in range(468):
        FEATURE_MATRIX[i][i]=1

    WEIGHTED_ADJACENCY_MATRIX=np.zeros((468,468)) #creating a numpy array of shape 468X468 initialized with zero
    BINARY_ADJACENCY_MATRIX=np.zeros((468,468))

    for edge in connection_tesselation:
        # Determining X & Y axis of Two Connected Points
        x1=x_axis[edge[0]]
        y1=y_axis[edge[0]]
        z1=z_axis[edge[0]]
        x2=x_axis[edge[1]]
        y2=y_axis[edge[1]]
        z2=z_axis[edge[1]]
        # Calculating Eucleadin Distance for Weighted Graph
        eucleadian_distance=(((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2) ** 0.5)
        # Creating Weighted Graph
        WEIGHTED_ADJACENCY_MATRIX[edge[0]][edge[1]]=eucleadian_distance
        WEIGHTED_ADJACENCY_MATRIX[edge[1]][edge[0]]=eucleadian_distance

        BINARY_ADJACENCY_MATRIX[edge[0]][edge[1]]=1
        BINARY_ADJACENCY_MATRIX[edge[1]][edge[0]]=1
        # Normalizing axis Values For Visualizing Meash on Face
        shape = image.shape 
        relative_x = int(x1 * shape[1])
        relative_y = int(y1 * shape[0])
        relative_z = int(x2 * shape[1])
        relative_c = int(y2 * shape[0])
        # Drawing Nodes and Edges on Image For Visualization
        cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 0, 255), thickness=3) 
        cv2.line(image,(relative_x,relative_y),(relative_z,relative_c),(0,255,0),1)

    # plt.imshow(image)
    WEIGHTED_ADJACENCY_MATRIX=sparse.csr_matrix(WEIGHTED_ADJACENCY_MATRIX)
    BINARY_ADJACENCY_MATRIX=sparse.csr_matrix(BINARY_ADJACENCY_MATRIX)
    # print(BINARY_ADJACENCY_MATRIX)
    LocalAngles = local_angles(BINARY_ADJACENCY_MATRIX, x_axis, y_axis, z_axis)
    # print(LocalAngles)
    XCoordinateFeature, YCoordinateFeature, ZCoordinateFeature = coordinateFeature(x_axis, y_axis, z_axis)
    # FEATURE_MATRIX=torch.from_numpy(FEATURE_MATRIX)
    DOTPRODUCT=WEIGHTED_ADJACENCY_MATRIX.dot(BINARY_ADJACENCY_MATRIX)
    # newFeatureMatrix = LocalAngles
    newFeatureMatrix = torch.from_numpy(np.column_stack((LocalAngles,  XCoordinateFeature[:-10], YCoordinateFeature[:-10], ZCoordinateFeature[:-10], FEATURE_MATRIX)))
    # print( LocalAngles[0], XCoordinateFeature[:-10][0], YCoordinateFeature[:-10][0], ZCoordinateFeature[:-10][0])
    return(WEIGHTED_ADJACENCY_MATRIX,newFeatureMatrix)
    # return(DOTPRODUCT,newFeatureMatrix/np.linalg.norm(newFeatureMatrix))


Checking mediapipe is running currectly or not

In [22]:
# path="./datasets/orl_dataset/person10/test_image/98_10.jpg"
# adj1,features1=mediapipe(path)
# for i in range(50):
#     print(features1[i])



tensor([-0.7759, -0.0235,  0.5058, -0.4985], dtype=torch.float64)
tensor([-0.5424,  0.0069,  0.3426, -0.9304], dtype=torch.float64)
tensor([ 0.6072, -0.0030,  0.3774, -0.5853], dtype=torch.float64)
tensor([-0.4404, -0.0540,  0.0469, -0.8909], dtype=torch.float64)
tensor([-0.6990,  0.0128,  0.2676, -1.0000], dtype=torch.float64)
tensor([-0.7419,  0.0193,  0.1480, -0.9965], dtype=torch.float64)
tensor([-0.1445,  0.0319, -0.1702, -0.8136], dtype=torch.float64)
tensor([-0.2440, -0.5995, -0.2972, -0.3943], dtype=torch.float64)
tensor([ 0.0725,  0.0438, -0.3957, -0.8246], dtype=torch.float64)
tensor([ 0.0010,  0.0507, -0.5089, -0.9042], dtype=torch.float64)
tensor([-0.4583,  0.0676, -0.9941, -0.9860], dtype=torch.float64)
tensor([-0.6409, -0.0252,  0.5234, -0.4675], dtype=torch.float64)
tensor([-0.6022, -0.0280,  0.5377, -0.4196], dtype=torch.float64)
tensor([-0.7103, -0.0302,  0.5464, -0.3439], dtype=torch.float64)
tensor([-0.6592, -0.0333,  0.5456, -0.3258], dtype=torch.float64)
tensor([-0

In [23]:
# adj1=adj1.toarray()
# adj2=adj2.toarray()

In [24]:
# path="./datasets/orl_dataset/person9/test_image/88_9.jpg"
# adj1,features1=mediapipe(path)
# for i in range(50):
#     print(features1[i])
